In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

import config
import models.autoencoder
import models.cyclegan
import plot_loss_history
import util

plt.style.use('bmh')

%reload_ext autoreload
%autoreload 2

## Plot losses.

In [ ]:
model_key = '2020-05-22T23:58:01'

plot_loss_history.plot_losses(model_key, min_epoch=2, log=True, max_epoch=50)

In [ ]:
model_key = '2020-05-26T22:40:22'

plot_loss_history.plot_losses(model_key, min_epoch=2, log=True)

In [ ]:
model_key = '2020-05-27T03:21:01'

plot_loss_history.plot_losses(model_key, min_epoch=1, log=True)

In [ ]:
model_key = '2020-05-27T18:01:17'

plot_loss_history.plot_losses(model_key, min_epoch=1, log=True)

In [ ]:
model_key = '2020-05-27T19:09:56'

plot_loss_history.plot_losses(model_key, min_epoch=1, log=True)

In [ ]:
model_key = '2020-05-28T04:07:10'

plot_loss_history.plot_losses(model_key, min_epoch=50)

In [ ]:
model_key = '2020-05-28T12:28:08'

plot_loss_history.plot_losses(model_key, min_epoch=50)

In [ ]:
model_key = '2020-05-28T17:09:11'

plot_loss_history.plot_losses(model_key, min_epoch=50)

In [ ]:
model_key = '2020-05-28T18:37:49'

plot_loss_history.plot_losses(model_key, min_epoch=50)

## Inspect model architecture.

In [ ]:
model_config = config.cyclegan_config_from_json('panzer_schaeferhund.json')
model = models.cyclegan.CycleGAN(model_config)

In [ ]:
model.D['A'].summary()

In [ ]:
util.plot_model_svg(model.D_A, dpi=50, expand_nested=True, show_shapes=True)

In [ ]:
model.G_single['A'].summary()

In [ ]:
util.plot_model_svg(model.G_single['A'], dpi=50, show_shapes=True)

In [ ]:
model.G.summary()

In [ ]:
model_config = config.autoencoder_config_from_json('panzer_schaeferhund_autoencoder_adain.json')
model = models.autoencoder.AutoEncoder(model_config)

print(model.encoder.count_params())
print(model.decoder.count_params())

display(util.plot_model_svg(model.encoder, dpi=40, show_shapes=True))
display(util.plot_model_svg(model.decoder, dpi=40, show_shapes=True))

In [ ]:
model_config = config.autoencoder_config_from_json('panzer_schaeferhund_autoencoder_cnn_sampling.json')
model = models.autoencoder.AutoEncoder(model_config)

print(model.encoder.count_params())
print(model.decoder.count_params())

display(util.plot_model_svg(model.encoder, dpi=40, show_shapes=True))
display(util.plot_model_svg(model.decoder, dpi=40, show_shapes=True))